1- Apply an explicit method. 


_Firstly, we have the "forward-backward differencing" (second-order centered differencing) of the viscous Burger's equations written as_
$$\frac{u^{n+1}_j-u^{n}_j}{\Delta t} =\frac{\frac{u^{n}_{j+1}-u^{n}_{j}}{\Delta x}-\frac{u^{n}_{j}-u^{n}_{j-1}}{\Delta x}}{\Delta x}=\frac{u^{n}_{j+1}-2u^{n}_{j}+u^{n}_{j-1}}{\Delta^2 x}$$
$$\Rightarrow u^{n+1}_j=u^{n}_j+\frac{\nu\Delta t}{\Delta^2 x}\left(u^{n}_{j+1}-2u^{n}_{j}+u^{n}_{j-1}\right)$$
_Then, the Von Neumann analysis of this method can be given as_
$$\xi^{n+1}e^{ikj\Delta x} = \xi^n e^{ikj\Delta x}+\frac{\nu\Delta t}{\Delta^2 x}(\xi^ne^{ik(j+1)\Delta x} - 2\xi^ne^{ikj\Delta x} + \xi^ne^{ik(j-1)\Delta x})$$
$$= \xi^n e^{ikj\Delta x} + \frac{\nu\Delta t}{\Delta^2 x}(\xi^ne^{ikj\Delta x}e^{ik\Delta x} - 2\xi^ne^{ikj\Delta x} + \xi^ne^{ikj\Delta x}e^{-ik\Delta x})$$
$$= \xi^n e^{ikj\Delta x} \left[1+\frac{\nu\Delta t}{\Delta^2 x}\left(e^{ik\Delta x}+e^{-ik\Delta x}-2\right)\right]$$
$$= \xi^n e^{ikj\Delta x} \left[1-2\frac{\nu\Delta t}{\Delta^2 x}\left(1-\cos(k\Delta x)\right)\right]$$
_Therefore, to guarantee a stable numerical solution, it is necessary to have $\Delta t\le 0.5\frac{\Delta^2 x}{\nu}$. Numerical tests show that the time steps needed are proportional to the squre of nump, and the computations will become unstable if CFL > 0.5._

In [41]:
import sys 
import os   
sys.path.append(os.getcwd()+'/..')
import numpy as np
import math
import matplotlib.pyplot as plt 
import nm_lib as nm
from typing import Type
from matplotlib import animation
import matplotlib
import time
from scipy.optimize import curve_fit
matplotlib.use('TkAgg')

# --- Values reused in other tasks of ex_3 ---
x0 = -2.6 
xf = 2.6
xc = 0.0
A = 0.3
W = 0.1
pi = math.pi
CFL = 0.5
nu = 1
# ---------------------------

def tfunc_Burgers(xx, A, W):
    return A * math.exp(-(xx - xc) ** 2.0 / W ** 2.0)

def discretisation(x0, xf, nump, dt_float: Type[float]):
    nint = nump - 1
    dh = dt_float((xf - x0) / nint)
    xx = dt_float(np.arange(nump) * dh + x0) 
    xx_half = dt_float(np.arange(nump) * dh + x0 + dh * 0.5)
    return nint, dh, xx, xx_half

def run_Burgers_explicit(nump, tf, CFL, A, W, nu, ddx1 = lambda x,y: nm.deriv_fwd(x,y), ddx2 = lambda x,y: nm.deriv_bck(x,y), 
                            bnd_limits: list=[1,1],):
    nint, dh, xx, xx_half = discretisation(x0, xf, nump, dt_float=np.float64)

    hh = np.zeros(nump)
    for i in range(0, nump):
        hh[i] = tfunc_Burgers(xx[i], A, W)

    dt = CFL * nm.cfl_diff_burger(nu, xx)
    nt = int(tf / dt) + 1 
    current_t = 0.0
    unnt = np.zeros((xx.size, nt)) 
    unnt[:, 0] = hh
    t = np.zeros(nt)  
    
    for i in range(0, nt-1):  
        up1 = nm.step_diff_burgers(xx, unnt[:, i], nu, ddx1)
        up1 = np.pad(up1[bnd_limits[0]:xx.size-bnd_limits[1]], bnd_limits, 'wrap') 
        up2 = nm.step_diff_burgers(xx, up1,        nu, ddx2)
        up2 = np.pad(up2[bnd_limits[0]:xx.size-bnd_limits[1]], bnd_limits, 'wrap') 
        
        for j in range(0, xx.size):
            unnt[j, i+1] = unnt[j, i] + dt * nu * up2[j]
        t[i+1] = t[i] + dt     
    
    plt.xlim(xx[0], xx[nump-1])
    plt.ylim(unnt[:, 0].min() - 0.1 * (unnt[:, 0].max() - unnt[:, 0].min()), 
             unnt[:, 0].max() + 0.1 * (unnt[:, 0].max() - unnt[:, 0].min()))
    plt.xlabel("x")
    plt.ylabel("u")
    
    line, = plt.plot([], [], '--', label='2nd-order centered differencing solution')
    legend = plt.legend(loc='lower right')
    time_text = plt.text(0.02, 0.95, '', transform=plt.gca().transAxes)

    def init():
        line.set_data([], [])
        time_text.set_text('')
        return line,  
 
    def update(frame):
        line.set_data(xx, unnt[:, frame])
        time_text.set_text("Time = " + str(t[frame]))
        return line,  

    anim = animation.FuncAnimation(plt.gcf(), update, init_func=init, frames=t.size-1, blit=True)
    
    plt.show(block=True)

    print(str(nt) + " time steps are needed to reach physical time " + str(tf) + " when " + str(nump) + " grid points are used.")
 
nump = 129
tf = 0.45 # For simplicity, 1/4 * 1.8 physical time is tested but it can be changed straigthforwardly. 
#run_Burgers_explicit(nump, tf, CFL, A, W, nu)
#nump = 257 
#tf = 0.45   
#run_Burgers_cent_2ndOrd(nump, tf, CFL, A, W, nu)

2- Implicit methods.

The implicit method is stable with large time steps, and thus it needs less computing time when (much) fewer steps are used. The iterations needed depend on both the "toll" parameter of `Newton_Raphson` and the number of time steps.

In [ ]:
# Explicit and implicit method solving the linear equation

def tfunc_SelfSimilar_linear(xx, u0, nu, phi, t): 
    Rs0 = phi / math.sqrt(2.0 * pi) / u0
    chi = 2.0 * nu / (Rs0 * Rs0)
    Rst = Rs0 * math.sqrt(1.0 + chi * t)  
    
    return u0 / math.sqrt(1.0 + chi * t) * math.exp(-(xx * xx) / (2.0 * Rst * Rst))

def fit_unnt(xx, unnt, t, label):
    t_last = t[-1]  # Get the last time value
    y_data = unnt[:, -1]  # Get the values at the last time step

    # Initial guess for u0, nu, phi
    p0 = [1.5, 0.5, 1.5]

    # Define the bounds for the parameters
    bounds = ([0, 0, 0], [2.0, 2.0, 2.0])

    # Perform the curve fitting
    popt, pcov = curve_fit(
        # Key change: This lambda applies `tfunc_SelfSimilar_linear` to each element of `xx`
        lambda xx, u0, nu, phi: np.array([tfunc_SelfSimilar_linear(x, u0, nu, phi, t_last) for x in xx]),  
        xx, y_data, p0=p0, bounds=bounds, xtol=1e-8, 
    )

    # popt will contain the optimized parameters u0, nu, phi
    u0_opt, nu_opt, phi_opt = popt

    # Plot the fitted curve
    plt.figure(figsize=(8, 5))
    plt.plot(xx, y_data, 'o', label="Numerical data (last step)", alpha=0.6)
    plt.plot(xx, np.array([tfunc_SelfSimilar_linear(x, u0_opt, nu_opt, phi_opt, t_last) for x in xx]), '-', label="Fitted model")
    plt.xlabel('x')
    plt.ylabel('u')
    plt.legend()
    plt.title(f"Fitting result for {label}")
    plt.show()

    # Print the optimized parameters
    print(f"Optimized parameters for {label}:")
    print(f"u0 = {u0_opt:.4f}, nu = {nu_opt:.4f}, phi = {phi_opt:.4f}")

def run_SelfSimilar_Implicit(nump, ti, tf, nt, u0, nu, phi, ncount=20, bnd_limits=[1, 1]):
    nint, dh, xx, xx_half = discretisation(x0, xf, nump, dt_float=np.float64)

    hh = np.zeros(nump)
    for i in range(0, nump):
        hh[i] = tfunc_SelfSimilar_linear(xx[i], u0, nu, phi, ti)

    start_time_implicit = time.time()
    dt = (tf - ti) / float(nt - 1)
    t, unnt, errt, countt = nm.Newton_Raphson(xx, hh, nu, dt, nt, toll=1e-4, ncount=ncount)
    end_time_implicit = time.time()
    print(f"Implicit function execution time: {end_time_implicit - start_time_implicit:.6f} seconds")
    print(f"Implicit function maximum iteration: {countt}")

    t = np.zeros((nt))
    unnt_selfSimilar = np.zeros((np.size(xx), nt))
    for i in range(0, nt):
        for j in range(0, np.size(xx)):
            unnt_selfSimilar[j, i] = tfunc_SelfSimilar_linear(xx[j], u0, nu, phi, ti + i * dt)  
        t[i] = ti + i * dt
            
    plt.figure()
    plt.xlim(xx[0], xx[nump - 1])
    plt.ylim(
        unnt_selfSimilar[:, 0].min() - 0.1 * (unnt_selfSimilar[:, 0].max() - unnt_selfSimilar[:, 0].min()),
        unnt_selfSimilar[:, 0].max() + 0.1 * (unnt_selfSimilar[:, 0].max() - unnt_selfSimilar[:, 0].min())
    )
    plt.xlabel("x")
    plt.ylabel("u") 

    line1, = plt.plot([], [], '--', label='Fully implicit solution')
    line2, = plt.plot([], [], '--', label='Self-similar solution')
    plt.legend(loc='lower right')

    time_text = plt.text(0.02, 0.95, '', transform=plt.gca().transAxes)
 
    def init():
        line1.set_data([], [])
        line2.set_data([], [])
        time_text.set_text('')
        return line1, line2,

    def update(frame):
        line1.set_data(xx, unnt[:, frame])
        line2.set_data(xx, unnt_selfSimilar[:, frame])
        time_text.set_text("Time = " + str(t[frame]))
        return line1, line2,

    anim = animation.FuncAnimation(plt.gcf(), update, init_func=init, frames=t.size - 1, blit=True)

    plt.show(block=True)

    fit_unnt(xx, unnt, t, "Implicit")

def run_SelfSimilar_explicit(nump, ti, tf, CFL, u0, nu, phi, ddx1 = lambda x,y: nm.deriv_fwd(x,y), ddx2 = lambda x,y: nm.deriv_bck(x,y), 
                            bnd_limits: list=[1,1],):
    nint, dh, xx, xx_half = discretisation(x0, xf, nump, dt_float=np.float64)

    hh = np.zeros(nump)
    for i in range(0, nump):
        hh[i] = tfunc_SelfSimilar_linear(xx[i], u0, nu, phi, ti)

    dt = CFL * nm.cfl_diff_burger(nu, xx)
    nt = int(tf / dt) + 1  
    unnt = np.zeros((xx.size, nt)) 
    unnt[:, 0] = hh
    t = np.zeros(nt) 
    
    start_time_explicit = time.time()
    unnt_selfSimilar = np.zeros((np.size(xx), nt))
    for i in range(0, nt-1):  
        up1 = nm.step_diff_burgers(xx, unnt[:, i], nu, ddx1)
        up1 = np.pad(up1[bnd_limits[0]:xx.size-bnd_limits[1]], bnd_limits, 'wrap') 
        up2 = nm.step_diff_burgers(xx, up1,        nu, ddx2)
        up2 = np.pad(up2[bnd_limits[0]:xx.size-bnd_limits[1]], bnd_limits, 'wrap') 
        
        for j in range(0, xx.size):
            unnt[j, i+1] = unnt[j, i] + dt * nu * up2[j]
            unnt_selfSimilar[j, i] = tfunc_SelfSimilar_linear(xx[j], u0, nu, phi, ti + i * dt)  
        t[i+1] = t[i] + dt  

    end_time_explicit = time.time()
    print(f"Explicit function execution time: {end_time_explicit - start_time_explicit:.6f} seconds")

    #print(t, t.size)

    plt.figure()
    plt.xlim(xx[0], xx[nump - 1])
    plt.ylim(
        unnt_selfSimilar[:, 0].min() - 0.1 * (unnt_selfSimilar[:, 0].max() - unnt_selfSimilar[:, 0].min()),
        unnt_selfSimilar[:, 0].max() + 0.1 * (unnt_selfSimilar[:, 0].max() - unnt_selfSimilar[:, 0].min())
    )
    plt.xlabel("x")
    plt.ylabel("u") 

    line1, = plt.plot([], [], '--', label='Explicit solution')
    line2, = plt.plot([], [], '--', label='Self-similar solution')
    plt.legend(loc='lower right')

    time_text = plt.text(0.02, 0.95, '', transform=plt.gca().transAxes)
 
    def init():
        line1.set_data([], [])
        line2.set_data([], [])
        time_text.set_text('')
        return line1, line2,

    def update(frame):
        line1.set_data(xx, unnt[:, frame])
        line2.set_data(xx, unnt_selfSimilar[:, frame])
        time_text.set_text("Time = " + str(t[frame]))
        return line1, line2,

    anim = animation.FuncAnimation(plt.gcf(), update, init_func=init, frames=t.size - 1, blit=True)

    plt.show(block=True)

    fit_unnt(xx, unnt, t, "Explicit")

# Parameters for the simulation
nump = 129
ti = 0.15
tf = 0.45
nt = 21
phi = 1.0
u0 = 1.0
nu = 1.0
ncount = 40
#run_SelfSimilar_Implicit(nump, ti, tf, nt, u0, nu, phi, ncount=ncount)
CFL = 0.5 
run_SelfSimilar_explicit(nump, ti, tf, CFL, u0, nu, phi)

Explicit function execution time: 0.184343 seconds
[0.00000000e+00 4.12597656e-04 8.25195312e-04 ... 4.48906250e-01
 4.49318848e-01 4.49731445e-01] 1091


invalid command name "6140292416process_stream_events"
    while executing
"6140292416process_stream_events"
    ("after" script)
can't invoke "event" command: application has been destroyed
    while executing
"event generate $w <<ThemeChanged>>"
    (procedure "ttk::ThemeChanged" line 6)
    invoked from within
"ttk::ThemeChanged"


Optimized parameters for Explicit:
u0 = 0.7157, nu = 1.2845, phi = 1.0314


In [43]:
#Implicit method solving the nonlinear equation

#def tfunc_Init_u(xx, a, phi, t): 
#    C = 4.0 * pi * a * t
#    R = 3.0 / 2.0 * (C * (phi / pi) ** 2.0)
#    if xx ** 2.0 < R ** 2.0:
#        return (phi / C) ** (1.0 / 2.0) * (1.0 - xx ** 2.0 / R ** 2.0)
#    else:
#        return 0.0

def run_Implicit_u(nump, ti, tf, nt, u0, nu, phi, ncount=20, bnd_limits=[1, 1]):
    nint, dh, xx, xx_half = discretisation(x0, xf, nump, dt_float=np.float64)

    hh = np.zeros(nump)
    for i in range(0, nump):
        #hh[i] = tfunc_Init_u(xx[i], nu, phi, ti) # 
        hh[i] = tfunc_SelfSimilar_linear(xx[i], u0, nu, phi, ti)

    start_time_implicit = time.time()
    dt = (tf - ti) / float(nt - 1)
    t, unnt, errt, countt = nm.Newton_Raphson_u(xx, hh, dt, nt, toll=1e-4, ncount=ncount)
    end_time_implicit = time.time()
    print(f"Implicit function execution time: {end_time_implicit - start_time_implicit:.6f} seconds")
    print(f"Implicit function maximum iteration: {countt}")
            
    plt.figure()
    plt.xlim(xx[0], xx[nump - 1])
    plt.ylim(
        unnt[:, 0].min() - 0.1 * (unnt[:, 0].max() - unnt[:, 0].min()),
        unnt[:, 0].max() + 0.1 * (unnt[:, 0].max() - unnt[:, 0].min())
    )
    plt.xlabel("x")
    plt.ylabel("u") 

    line, = plt.plot([], [], '--', label='Fully implicit nonlinear solution')
    plt.legend(loc='lower right')

    time_text = plt.text(0.02, 0.95, '', transform=plt.gca().transAxes)
 
    def init():
        line.set_data([], []) 
        time_text.set_text('')
        return line,

    def update(frame):
        line.set_data(xx, unnt[:, frame]) 
        time_text.set_text("Time = " + str(t[frame]))
        return line,  

    anim = animation.FuncAnimation(plt.gcf(), update, init_func=init, frames=t.size - 1, blit=True)

    plt.show(block=True)

nump = 129
ti = 0.05
tf = 0.45
nt = 21
phi = 1.0
u0 = 1.0
nu = 1.0
ncount = 1000
#run_Implicit_u(nump, ti, tf, nt, u0, nu, phi, ncount=ncount)



3- Semi-explicit methods. 

In [44]:
nu_values = np.linspace(0.5, 1.0, 100) 
niter_values = np.arange(1, 6)  
 
fig, axes = plt.subplots(len(niter_values), 1, figsize=(8, len(niter_values) * 4), sharex=True)
 
for idx, niter in enumerate(niter_values):
    ax = axes[idx]
    taui_sum_values = np.zeros_like(nu_values)
    for iiter in np.arange(1, niter + 1):
        taui_values = np.array([nm.taui_sts(nu, niter, iiter) for nu in nu_values])
        ax.plot(nu_values, taui_values, label=f'iiter={iiter}')
        taui_sum_values += taui_values
    
    ax.plot(nu_values, taui_sum_values, label='Summation of taui', linestyle='--', color='black')
    #ax.set_yscale('log')
    ax.set_title(f"taui vs nu for niter={niter}")
    ax.set_xlabel("nu")
    ax.set_ylabel("taui")
    ax.legend()
    ax.grid(True) 
plt.tight_layout()
plt.show()

can't invoke "event" command: application has been destroyed
    while executing
"event generate $w <<ThemeChanged>>"
    (procedure "ttk::ThemeChanged" line 6)
    invoked from within
"ttk::ThemeChanged"


In [45]:
def run_SelfSimilar_STS(nump, ti, nt, a, CFL, phi):
    nint, dh, xx, xx_half = discretisation(x0, xf, nump, dt_float=np.float64)

    hh = np.zeros(nump)
    for i in range(0, nump):
        hh[i] = tfunc_SelfSimilar_linear(xx[i], u0, a, phi, ti)

    start_time_sts = time.time() 
    t, unnt = nm.evol_sts(xx, hh, nt, a, cfl_cut=CFL)
    end_time_sts = time.time()
    print(f"STS function execution time: {end_time_sts - start_time_sts:.6f} seconds")
    print(t)

    unnt_selfSimilar = np.zeros((np.size(xx), nt))
    for i in range(0, nt):
        for j in range(0, np.size(xx)):
            unnt_selfSimilar[j, i] = tfunc_SelfSimilar_linear(xx[j], u0, a, phi, ti+t[i])  
            
    plt.figure()
    plt.xlim(xx[0], xx[nump - 1])
    plt.ylim(
        unnt_selfSimilar[:, 0].min() - 0.1 * (unnt_selfSimilar[:, 0].max() - unnt_selfSimilar[:, 0].min()),
        unnt_selfSimilar[:, 0].max() + 0.1 * (unnt_selfSimilar[:, 0].max() - unnt_selfSimilar[:, 0].min())
    )
    plt.xlabel("x")
    plt.ylabel("u") 

    line1, = plt.plot([], [], '--', label='STS solution')
    line2, = plt.plot([], [], '--', label='Self-similar solution')
    plt.legend(loc='lower right')

    time_text = plt.text(0.02, 0.95, '', transform=plt.gca().transAxes)
 
    def init():
        line1.set_data([], [])
        line2.set_data([], [])
        time_text.set_text('')
        return line1, line2,

    def update(frame):
        line1.set_data(xx, unnt[:, frame])
        line2.set_data(xx, unnt_selfSimilar[:, frame])
        time_text.set_text("Time = " + str(t[frame]))
        return line1, line2,

    anim = animation.FuncAnimation(plt.gcf(), update, init_func=init, frames=t.size - 1, blit=True)

    plt.show(block=True)

nump = 129
ti = 0.05 
nt = 21 
a = 1.0 
CFL = 0.5
run_SelfSimilar_STS(nump, ti, nt, a, CFL, phi)

STS function execution time: 0.020134 seconds
[0.         0.00217458 0.00434916 0.00652374 0.00869832 0.0108729
 0.01304748 0.01522206 0.01739664 0.01957123 0.02174581 0.02392039
 0.02609497 0.02826955 0.03044413 0.03261871 0.03479329 0.03696787
 0.03914245 0.04131703 0.04349161]


invalid command name "6140239552process_stream_events"
    while executing
"6140239552process_stream_events"
    ("after" script)
can't invoke "event" command: application has been destroyed
    while executing
"event generate $w <<ThemeChanged>>"
    (procedure "ttk::ThemeChanged" line 6)
    invoked from within
"ttk::ThemeChanged"
